In [1]:

# coding: utf-8

# In[ ]:


from __future__ import division


# In[ ]:


import tensorflow as tf
import numpy as np
import matplotlib.pyplot as pyplot
from sklearn.model_selection import train_test_split
import cv2
import os
from skimage import io as ski_io
from skimage.viewer import ImageViewer
from skimage import color as ski_color
from skimage import io as ski_io
import time
from nolearn.lasagne import BatchIterator
from skimage import transform as ski_transform
NUM_CLASS=12
def one_hot(labels, n_class = NUM_CLASS):
    total_record = len(labels)
    y = np.zeros((total_record, n_class))
    y[np.arange(total_record), labels] = 1
    return y


from collections import namedtuple

Parameters = namedtuple('Parameters', [
        # Data parameters
        'num_classes', 'image_size', 
        # Training parameters
        'batch_size', 'max_epochs', 'log_epoch', 'print_epoch',
        # Optimisations
        'learning_rate_decay', 'learning_rate',
        'l2_reg_enabled', 'l2_lambda', 
        'early_stopping_enabled', 'early_stopping_patience', 
        'resume_training', 
        # Layers architecture
        'conv1_k', 'conv1_d', 'conv1_p', 
        'conv2_k', 'conv2_d', 'conv2_p', 
        'conv3_k', 'conv3_d', 'conv3_p', 
        'fc4_size', 'fc4_p'
    ])

import os

class Paths(object):
    """
    Provides easy access to common paths we use for persisting 
    the data associated with model training.
    """
    
    def __init__(self, params):
        """
        Initialises a new `Paths` instance and creates corresponding folders if needed.

        Parameters
        ----------
        params  : Parameters
                  Structure (`namedtuple`) containing model parameters.
        """
        self.model_name = self.get_model_name(params)
        self.var_scope = self.get_variables_scope(params)
        self.root_path = os.getcwd() + "/models/" + self.model_name + "/"
        self.model_path = self.get_model_path()
        self.train_history_path = self.get_train_history_path()
        self.learning_curves_path = self.get_learning_curves_path()
        if not os.path.exists(self.root_path):
            os.makedirs(self.root_path)

    def get_model_name(self, params):
        """
        Generates a model name with some of the crucial model parameters encoded into the name.

        Parameters
        ----------
        params  : Parameters
                  Structure (`namedtuple`) containing model parameters.
                  
        Returns
        -------
        Model name.
        """
        # We will encode model settings in its name: architecture, optimisations applied, etc.
        print ("call get model name")
        model_name = "k{}d{}p{}_k{}d{}p{}_k{}d{}p{}_fc{}p{}".format(
            params.conv1_k, params.conv1_d, params.conv1_p, 
            params.conv2_k, params.conv2_d, params.conv2_p, 
            params.conv3_k, params.conv3_d, params.conv3_p, 
            params.fc4_size, params.fc4_p
        )
        model_name += "_lrdec" if params.learning_rate_decay else "_no-lrdec"
        model_name += "_l2" if params.l2_reg_enabled else "_no-l2"
        return model_name

    def get_variables_scope(self, params):
        """
        Generates a model variable scope with some of the crucial model parameters encoded.

        Parameters
        ----------
        params  : Parameters
                  Structure (`namedtuple`) containing model parameters.
                  
        Returns
        -------
        Variables scope name.
        """
        # We will encode model settings in its name: architecture, optimisations applied, etc.
        var_scope = "k{}d{}_k{}d{}_k{}d{}_fc{}_fc0".format(
            params.conv1_k, params.conv1_d,
            params.conv2_k, params.conv2_d,
            params.conv3_k, params.conv3_d, 
            params.fc4_size
        )
        return var_scope

    def get_model_path(self):
        """
        Generates path to the model file.
   
        Returns
        -------
        Model file path.
        """
        return self.root_path + "model.ckpt"

    def get_train_history_path(self):
        """
        Generates path to the train history file.
   
        Returns
        -------
        Train history file path.
        """
        return self.root_path + "train_history"
    
    def get_learning_curves_path(self):
        """
        Generates path to the learning curves graph file.
   
        Returns
        -------
        Learning curves file path.
        """
        return self.root_path + "learning_curves.png"


# In[ ]:


def fully_connected(input, size):
    """
    Performs a single fully connected layer pass, e.g. returns `input * weights + bias`.
    """
    weights = tf.get_variable( 'weights', 
        shape = [input.get_shape()[1], size],
        initializer = tf.contrib.layers.xavier_initializer()
      )
    biases = tf.get_variable( 'biases',
        shape = [size],
        initializer = tf.constant_initializer(0.0)
      )
    return tf.matmul(input, weights) + biases

def fully_connected_relu(input, size):
    return tf.nn.relu(fully_connected(input, size))

def conv_relu(input, kernel_size, depth):
    """
    Performs a single convolution layer pass.
    """
    weights = tf.get_variable( 'weights', 
        shape = [kernel_size, kernel_size, input.get_shape()[3], depth],
        initializer = tf.contrib.layers.xavier_initializer()
      )
    biases = tf.get_variable( 'biases',
        shape = [depth],
        initializer = tf.constant_initializer(0.0)
      )
    conv = tf.nn.conv2d(input, weights,
        strides = [1, 1, 1, 1], padding = 'SAME')
    return tf.nn.relu(conv + biases)

def pool(input, size):
    """
    Performs a max pooling layer pass.
    """
    return tf.nn.max_pool(
        input, 
        ksize = [1, size, size, 1], 
        strides = [1, size, size, 1], 
        padding = 'SAME'
    )

def model_pass(input, params, is_training):
    # Convolutions

    with tf.variable_scope('conv1'):
        conv1 = conv_relu(input, kernel_size = params.conv1_k, depth = params.conv1_d) 
    with tf.variable_scope('pool1'): 
        pool1 = pool(conv1, size = 2)
        pool1 = tf.cond(is_training, lambda: tf.nn.dropout(pool1, keep_prob = params.conv1_p), lambda: pool1)
    with tf.variable_scope('conv2'):
        conv2 = conv_relu(pool1, kernel_size = params.conv2_k, depth = params.conv2_d)
    with tf.variable_scope('pool2'):
        pool2 = pool(conv2, size = 2)
        pool2 = tf.cond(is_training, lambda: tf.nn.dropout(pool2, keep_prob = params.conv2_p), lambda: pool2)
    with tf.variable_scope('conv3'):
        conv3 = conv_relu(pool2, kernel_size = params.conv3_k, depth = params.conv3_d)
    with tf.variable_scope('pool3'):
        pool3 = pool(conv3, size = 2)
        pool3 = tf.cond(is_training, lambda: tf.nn.dropout(pool3, keep_prob = params.conv3_p), lambda: pool3)
    
    # Fully connected
    
    # 1st stage output
    pool1 = pool(pool1, size = 4)
    shape = pool1.get_shape().as_list()
    pool1 = tf.reshape(pool1, [-1, shape[1] * shape[2] * shape[3]])
    
    # 2nd stage output
    pool2 = pool(pool2, size = 2)
    shape = pool2.get_shape().as_list()
    pool2 = tf.reshape(pool2, [-1, shape[1] * shape[2] * shape[3]])    
    
    # 3rd stage output
    shape = pool3.get_shape().as_list()
    pool3 = tf.reshape(pool3, [-1, shape[1] * shape[2] * shape[3]])
    
    flattened = tf.concat([pool1, pool2, pool3], 1)
    
    with tf.variable_scope('fc4'):
        fc4 = fully_connected_relu(flattened, size = params.fc4_size)
        fc4 = tf.cond(is_training, lambda: tf.nn.dropout(fc4, keep_prob = params.fc4_p), lambda: fc4)
    with tf.variable_scope('out'):
        logits = fully_connected(fc4, size = params.num_classes)
    return logits


parameters = Parameters(
    # Data parameters
    num_classes = NUM_CLASS,
    image_size = (32, 32),
    # Training parameters
    batch_size = 256,
    max_epochs = 100,
    log_epoch = 1,
    print_epoch = 1,
    # Optimisations
    learning_rate_decay = False,
    learning_rate = 0.0001,
    l2_reg_enabled = True,
    l2_lambda = 0.0001,
    early_stopping_enabled = True,
    early_stopping_patience = 100,
    resume_training = True,
    # Layers architecture
    conv1_k = 5, conv1_d = 32, conv1_p = 0.9,
    conv2_k = 5, conv2_d = 64, conv2_p = 0.8,
    conv3_k = 5, conv3_d = 128, conv3_p = 0.7,
    fc4_size = 1024, fc4_p = 0.5
)

/opt/python-virtualenv/ml-python3/lib/python3.5/site-packages/skimage/viewer/__init__.py:6: UserWarning: Viewer requires Qt
  warn('Viewer requires Qt')
/opt/python-virtualenv/ml-python3/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
import cv2
import numpy as np

In [3]:
def image_resize(image, width = None, height = None, inter = cv2.INTER_AREA):
    # initialize the dimensions of the image to be resized and
    # grab the image size
    dim = None
    (h, w) = image.shape[:2]

    # if both the width and height are None, then return the
    # original image
    if width is None and height is None:
        return image

    # check to see if the width is None
    if width is None:
        # calculate the ratio of the height and construct the
        # dimensions
        r = height / float(h)
        dim = (int(w * r), height)

    # otherwise, the height is None
    else:
        # calculate the ratio of the width and construct the
        # dimensions
        r = width / float(w)
        dim = (width, int(h * r))

    # resize the image
    resized = cv2.resize(image, dim, interpolation = inter)

    # return the resized image
    return resized
def rotate(image, angle, center = None, scale = 1.0):
    (h, w) = image.shape[:2]

    if center is None:
        center = (w / 2, h / 2)

    # Perform the rotation
    M = cv2.getRotationMatrix2D(center, angle, scale)
    rotated = cv2.warpAffine(image, M, (w, h))

    return rotated

In [4]:
map_class = ['stop', 'turn_left', 'turn_right', 'prob_left', 'prob_right', 'no_entry', 'speed_limit', 'min_speed', 'end_min', 'unknown', 'left', 'right']
map_answer = []

In [5]:
# threshold
top_thres=40
bottom_thres=50
prev_check_thres=5
prev_w_thres=5
prev_h_thres=5
roi_id=0
rand_appear_w_thres=20
rand_appear_h_thres=20


In [9]:
import cv2
import numpy as np
cap = cv2.VideoCapture('./input-sample/MVI_1061.avi')
num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_rois = np.zeros([num_frames, 5, 5]) #num frame, max roi per frame, properties
sign_cnt = np.zeros([num_frames * 5])

# preperty

In [7]:
def backtrack(frame_id, x, y, w, h, depth):
    if depth >= prev_check_thres:
        return True
    if frame_id < 0:
        return False
    for frame_p in frame_rois[frame_id-1]:
        None

In [10]:
# Initialisation routines: generate variable scope, create logger, note start time.
paths = Paths(parameters)

# Build the graph
graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed at run time with a training minibatch.
    tf_x = tf.placeholder(tf.float32, shape = (None, parameters.image_size[0], parameters.image_size[1], 1))
    is_training = tf.constant(False)
    with tf.variable_scope(paths.var_scope):
        predictions = tf.nn.softmax(model_pass(tf_x, parameters, is_training))
        top_k_predictions = tf.nn.top_k(predictions, 5)

with tf.Session(graph = graph) as session:
    session.run(tf.global_variables_initializer())
    tf.train.Saver().restore(session, paths.model_path)        
    while (1):

        _, frame = cap.read()
        frame = cv2.resize(frame, (640, 360), interpolation = cv2.INTER_LINEAR)
        blur = cv2.medianBlur(frame.copy(), 5)

        # Convert BGR to HSV
        hsv = cv2.cvtColor(blur, cv2.COLOR_BGR2HSV)

        # define range of blue color in HSV
        lower_blue = np.array([100, 70, 40])
        upper_blue = np.array([130, 255, 255])

        lower_red = np.array([0, 70, 50])
        upper_red = np.array([10, 255, 255]) 
        r_mask1 = cv2.inRange(hsv, lower_red, upper_red) 
        r_mask2 = cv2.inRange(hsv, np.array([100, 10, 170]), np.array([180, 230, 255])) 
        r_mask3 = cv2.inRange(hsv, np.array([157, 60, 11]), np.array([180, 255, 255])) 
        red_mask = r_mask1 | r_mask2 | r_mask3
        

        # Threshold the HSV image to get only blue colors
        blue_mask = cv2.inRange(hsv, lower_blue, upper_blue)
        mask = blue_mask + red_mask

        # Bitwise-AND mask and original image
        res = cv2.bitwise_and(frame, frame, mask=mask)
        mask = cv2.erode(mask.copy(), None, iterations=2)
        mask2 = cv2.dilate(mask.copy(), None, iterations=2)

        cnts = cv2.findContours(mask2.copy(), cv2.RETR_EXTERNAL,
                                cv2.CHAIN_APPROX_SIMPLE)[-2]
        framedraw = frame.copy()
#         framedraw = cv2.cvtColor(framedraw, cv2.COLOR_BGR2GRAY)
#         framedraw = cv2.Canny(framedraw, 100, 200)
        if len(cnts) > 0:
            for cnt in cnts:
                x, y, w, h = cv2.boundingRect(cnt)
#                 Vi tri cac bien            
                if w < 20 or h < 20 or y < 30 or h / w > 2 or w / h > 1.5:
                    cv2.rectangle(framedraw, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    continue
                roi = frame[y:y+h, x:x+w]
                print (y, x, y +h, x + w)
                # extract image and feed to cnn
                roi = cv2.cvtColor(roi,cv2.COLOR_BGR2RGB)  
                roi = ski_transform.resize(roi, (32, 32))
                roi = ski_color.rgb2gray(roi)
                roi = roi.reshape(roi.shape + (1,))        
                tf_X = np.array([roi])
                cv2.rectangle(framedraw, (x, y), (x + w, y + h), (255,0,0), 2)
                [prediction] = session.run([top_k_predictions], feed_dict = {
                        tf_x : tf_X
                    }
                )
                prob_class =prediction[0][0][0] 
                class_ord = prediction[1][0][0]
                class_name = map_class[class_ord]
                cv2.putText(framedraw, class_name + "-" + str(prob_class), (x-100, y + int(h / 2)), cv2.FONT_HERSHEY_SIMPLEX, 1,color=(0,0,255))
        cv2.imshow('frame', framedraw)

        k = cv2.waitKey(10) & 0xFF
        if k == 27:
            break

cv2.destroyAllWindows()
cv2.VideoWriter()

call get model name
INFO:tensorflow:Restoring parameters from /home/tuyenlv17/data/programming/projects/digital-race-2/GSS-CNN-classification/models/k5d32p0.9_k5d64p0.8_k5d128p0.7_fc1024p0.5_no-lrdec_l2/model.ckpt
226 575 264 609
128 489 214 566
229 576 265 607
132 490 215 566
230 575 267 609
133 487 214 563


/opt/python-virtualenv/ml-python3/lib/python3.5/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


231 574 267 607
133 486 217 563
233 574 269 607
137 486 218 562
233 572 274 609
136 484 220 562
234 571 273 605
140 485 222 560
235 569 277 606
143 484 223 559
238 569 279 605
144 483 224 558
241 568 280 603
147 481 228 557
242 567 283 603
148 480 228 556
243 565 285 601
150 479 230 555
244 564 285 599
151 479 232 554
246 562 288 599
152 477 236 553
248 562 291 599
152 476 238 554
251 561 292 597
155 475 240 552
253 560 300 602
157 474 241 552
255 561 295 595
157 472 243 551
255 558 301 599
160 473 253 559
255 558 298 595
159 471 245 550
257 558 297 592
160 472 246 549
257 558 299 594
161 470 247 549
256 556 298 593
158 468 245 548
256 557 298 593
160 470 246 549
256 557 299 593
158 467 246 549
257 557 298 593
161 470 245 549
256 557 297 593
159 469 246 549
257 558 298 594
158 469 244 548
258 559 297 593
160 469 245 548
257 559 297 593
158 469 246 549
257 559 298 594
158 469 246 550
258 560 299 595
158 469 247 550
258 560 300 595
159 471 247 551
260 561 301 596
159 471 248 551
260 561 

209 597 237 628
148 516 210 599
211 602 241 633
149 516 214 602
216 606 244 636
150 518 216 608
219 611 248 640
150 515 219 612
222 616 242 640
149 516 222 616
149 516 224 620
148 518 226 623
147 517 229 626
148 517 232 634
148 518 236 640
148 518 236 640
149 518 237 640
148 518 241 640
148 520 242 640
150 521 243 640
148 521 249 640
149 523 249 640
149 523 240 633
151 526 244 635
150 525 247 638
151 526 240 636
152 529 236 635
150 527 238 640
151 530 238 640
153 535 227 633
150 526 229 637
152 530 217 618
159 541 220 621
160 542 223 624
162 545 201 600
161 546 203 605
164 551 206 610
165 553 207 613
167 555 207 612
168 556 210 616
170 559 211 615
172 561 211 615
174 564 207 609
174 565 209 611
176 568 207 606
179 572 207 605
138 491 159 518
140 491 160 519
141 494 163 522
144 496 166 525
146 497 169 527
149 500 171 528
152 502 175 531
154 503 178 533
156 505 180 535
90 262 110 282
157 506 182 537
159 507 183 537
161 509 185 540
163 511 186 539
166 513 190 541
168 515 191 543
170 517 1

66 416 103 441
191 430 296 514
154 151 256 248
65 414 103 438
191 431 289 512
156 150 260 247
65 414 102 437
266 399 296 428
192 431 290 512
158 149 261 245
68 415 104 442
30 409 54 431
266 382 301 428
193 431 287 510
165 149 262 238
68 415 105 443
193 433 291 511
159 148 265 235
68 415 105 443
193 433 287 508
161 147 257 242
68 415 105 443
280 382 312 405
192 432 287 510
163 146 259 238
67 414 104 442
192 429 287 510
165 145 260 229
85 419 105 440
193 430 285 507
167 144 261 229
83 416 106 441
194 431 289 508
174 145 266 235
72 409 107 439
194 432 295 508
175 144 269 234
73 413 107 439
194 432 294 506
176 144 267 233
72 411 107 440
34 407 54 433
195 431 282 505
176 142 270 232
73 410 108 441
196 430 285 503
179 141 268 229
277 378 303 415
197 428 290 503
178 141 269 218
76 416 103 436
281 387 309 417
197 428 293 502
181 140 268 216
75 416 102 436
283 389 309 418
277 37 300 65
198 426 290 504
190 139 274 216
173 427 206 449
75 413 102 435
288 392 310 418
197 428 286 504
191 139 275 216

error: /io/opencv/modules/imgproc/src/resize.cpp:4044: error: (-215) ssize.width > 0 && ssize.height > 0 in function resize
